**Assignment 3: Puma 560 Kinematics**

**Kashirin Aleksandr**

**Skoltech 2021**

**Task: Using Denavit-Hartenberg notation:**
* a) Find link parameters for the Denavit-Hartenberg table
* b) Find transformation matrices: $T^0_1, T^1_2, T^2_3, T^3_4, T^4_5, T^5_6$
* c) Find transformation matrix for ender position: $T^0_6$
* d) Find end-effector position

**I. Rules of frames implementation for Denavit-Hartenberg notation:**
* 1) The Z-axis is the axis of rotation for a revolute joint or an axis
* 2) The X-axis must be perpendicular to both the current Z-axis and the previous Z-axis.
* 3) The Y-axis is determined from X-axis and Z-axis by using right-hand-rule coordinate system.
* 4) The X-axis must intersect the previous Z-axis (Rule does not apply to frame 0).

**II. Denavit-Hartenberg notation:**
* $\alpha_{i-1} -$ is the angle from $Z_{i-1}$ to $Z_i$ measured about $X_{i-1}$ axis.
* $a_{i-1} -$ is the the distance from $Z_{i-1}$ to $Z_i$ measured along $X_{i-1}$ axis.
* $d_i -$ is the distance from $X_{i-1}$ to $X_i$ measured along $Z_i$ axis.
* $\theta_i -$ is the angle from $X_{i-1}$ to $X_i$ measured about $Z_i$ axis.

**III. Fill the table with link parameters. Amount of rows in this table should be determined as amount of assigned frames - 1.**

\begin{array}{c|c}
  i & \alpha_{i-1} & a_{i-1} & d_i & \theta_i \\ 
  \hline
  ... & ... & ... & ... & ... \\
  \hline
  N_{frames} - 1 & ... & ... & ... & ...
\end{array}

**IV. After determining link parameters using Denavit-Hartenberg notation, fill the transformation matrix:**
$$ T_i^{i-1} =  \begin{bmatrix} 
                  \cos{\theta_i} & -\sin{\theta_i} & 0 & a_{i-1} \\ 
                  \sin{\theta_i}\cos{\alpha_{i-1}} & \cos{\theta_i}\cos{\alpha_{i-1}} & -\sin{\alpha_{i-1}} & -\sin{\alpha_{i-1}}d_i \\ 
                  \sin{\theta_i}\sin{\alpha_{i-1}} & \cos{\theta_i}\sin{\alpha_{i-1}} & \cos{\alpha_{i-1}} &  \cos{\alpha_{i-1}}d_i \\
                  0 & 0 & 0 & 1
                \end{bmatrix}$$

**V. Find the transformation matrix for end-effector position:**
$$T_N^0 = T^0_1 T^1_2 T^2_3 T^3_4 ... T^{N-1}_N$$

![Puma 560](https://vuzlit.ru/imag_/8/4954/image001.png)

*Picture 1: Puma 560, Robot representation*

![title](https://d2t1xqejof9utc.cloudfront.net/screenshots/pics/3de1bda158ba2707a5554bc6909246fa/large.png)

*Picture 2: Assigned frames*

**Task a: Find link parameters**

We have 7 frames, hence the table will be consisted of 6 rows.

\begin{array}{c|c}
  i & \alpha_{i-1} & a_{i-1} & d_i & \theta_i \\ 
  \hline
  1 & 0 & 0 & 0 & \theta_1 \\
  \hline
  2 & -\frac{\pi}{2} & 0 & 0 & \theta_2 \\
  \hline
  3 & 0 & a_2 & d_3 & \theta_3 \\
  \hline
  4 & -\frac{\pi}{2} & a_3 & d_4 & \theta_4 \\
  \hline
  5 & \frac{\pi}{2} & 0 & 0 & \theta_5 \\
  \hline
  6 & -\frac{\pi}{2} & 0 & 0 & \theta_6 \\
  \hline
\end{array}

**Task b: Find transformation matrices: $T^0_1, T^1_2, T^2_3, T^3_4, T^4_5, T^5_6$**

In [11]:
# Import libraries
import sympy as sp
import numpy as np

# Define symbols
t1, t2, t3, t4, t5, t6, a2, a3, d3, d4 = sp.symbols('theta_1 theta_2 theta_3 theta_4 theta_5 theta_6 a_2 a_3 d_3 d_4')

# Initialize D-H table parameters
alpha = [0, -sp.pi/2, 0, -sp.pi/2, sp.pi/2, -sp.pi/2]
a = [0, 0, a2, a3, 0, 0]
d = [0, 0, d3, d4, 0, 0]
theta = [t1, t2, t3, t4, t5, t6]

# Create D-H table
DH_table = sp.Matrix([alpha, a, d, theta]).T

# Define the function to get transformation matrix
def get_tf_matrix(DH_table, i):
    """ Returns transformation matrix between i and i-1 frames

    Args:
        DH_table - Denavit–Hartenberg table of parameters
        i - Frame index

    Returns:
        T - Tranformation matrix
    """
    if (i - 1) >= 0:
        if ((i - 1) <= DH_table.shape[0]):

            # Assign the parameters
            alpha = DH_table[i - 1 , 0]
            a = DH_table[i - 1 , 1]
            d = DH_table[i - 1 , 2]
            theta = DH_table[i - 1 , 3]

            # Calculate transforamtion matrix
            T = sp.Matrix([[sp.cos(theta), -sp.sin(theta), 0, a],
                           [sp.sin(theta)*sp.cos(alpha), sp.cos(theta)*sp.cos(alpha), -sp.sin(alpha), -sp.sin(alpha)*d],
                           [sp.sin(theta)*sp.sin(alpha), sp.cos(theta)*sp.sin(alpha), sp.cos(alpha), sp.cos(alpha)*d],
                           [0, 0, 0, 1]])

            return sp.simplify(T)
        else:
            raise ValueError(f'Frame index should less or equal the maximum frame index. Maximum frame index: {DH_table.shape[0]}')
    else:
        raise ValueError('Frame index should be more than 0')

# Print T_0_1 - transforamtion matrix between 0 and 1 frames
get_tf_matrix(DH_table, 4)

Matrix([
[ cos(theta_4), -sin(theta_4), 0, a_3],
[            0,             0, 1, d_4],
[-sin(theta_4), -cos(theta_4), 0,   0],
[            0,             0, 0,   1]])

 **Task c: Find transformation matrix for ender position: $T^0_6$**

In [2]:
# Define a function for end-effector transformation matrix
def get_resulted_tf(DH_table):
    """ Returns resulted transforamation matrix for end-effector

    Args:
        DH_table - Denavit–Hartenberg table of parameters
    
    Returns:
        res_T - Resulted tranformation matrix
    """
    # Initialize list of all transforametion matrices
    res_T = 1
    for i in range(1, DH_table.shape[0] + 1):
        # Append new transforamtion matrix
        res_T *= get_tf_matrix(DH_table, i)
    return sp.trigsimp(res_T)

# Print the resulted transformation matrix
res_T = get_resulted_tf(DH_table)
res_T

Matrix([
[((sin(theta_1)*sin(theta_4) + cos(theta_1)*cos(theta_4)*cos(theta_2 + theta_3))*cos(theta_5) - sin(theta_5)*sin(theta_2 + theta_3)*cos(theta_1))*cos(theta_6) - (-sin(theta_1)*cos(theta_4) + sin(theta_4)*cos(theta_1)*cos(theta_2 + theta_3))*sin(theta_6), -((sin(theta_1)*sin(theta_4) + cos(theta_1)*cos(theta_4)*cos(theta_2 + theta_3))*cos(theta_5) - sin(theta_5)*sin(theta_2 + theta_3)*cos(theta_1))*sin(theta_6) - (-sin(theta_1)*cos(theta_4) + sin(theta_4)*cos(theta_1)*cos(theta_2 + theta_3))*cos(theta_6), -(sin(theta_1)*sin(theta_4) + cos(theta_1)*cos(theta_4)*cos(theta_2 + theta_3))*sin(theta_5) - sin(theta_2 + theta_3)*cos(theta_1)*cos(theta_5), a_2*cos(theta_1)*cos(theta_2) + a_3*cos(theta_1)*cos(theta_2 + theta_3) - d_3*sin(theta_1) - d_4*sin(theta_2 + theta_3)*cos(theta_1)],
[ ((sin(theta_1)*cos(theta_4)*cos(theta_2 + theta_3) - sin(theta_4)*cos(theta_1))*cos(theta_5) - sin(theta_1)*sin(theta_5)*sin(theta_2 + theta_3))*cos(theta_6) - (sin(theta_1)*sin(theta_4)*cos(theta_2 

**Task d: Find end-effector position**

In [3]:

# Define a function to find an end-effector position
def find_ef_pos(res_T):
    """ Returns end-effector position

    Args:
        res_T - Resulted tranformation matrix
    
    Returns:
        pos - End-effector position
    """
    pos = res_T[:3, 3]
    return sp.trigsimp(pos)
    
# Print end-effector position
pos = find_ef_pos(res_T)
pos

Matrix([
[a_2*cos(theta_1)*cos(theta_2) + a_3*cos(theta_1)*cos(theta_2 + theta_3) - d_3*sin(theta_1) - d_4*sin(theta_2 + theta_3)*cos(theta_1)],
[a_2*sin(theta_1)*cos(theta_2) + a_3*sin(theta_1)*cos(theta_2 + theta_3) + d_3*cos(theta_1) - d_4*sin(theta_1)*sin(theta_2 + theta_3)],
[                                                         -a_2*sin(theta_2) - a_3*sin(theta_2 + theta_3) - d_4*cos(theta_2 + theta_3)]])

**Get the numerical answer**

In [4]:
# Initialize parameters
params = (t1, t2, t3, t4, t5, t6, a2, a3, d3, d4)
# Define conversation function for D-H table
dh_sym2num = sp.lambdify(params, DH_table, modules='numpy')

# Set numerical parameters
theta_1 = np.deg2rad(45)  # Angle between 0 and 1 frames
theta_2 = np.deg2rad(60)  # Angle between 1 and 2 frames
theta_3 = np.deg2rad(75)  # Angle between 2 and 3 frames
theta_4 = 0  # Angle between 3 and 4 frames
theta_5 = 0  # Angle between 4 and 5 frames
theta_6 = 0  # Angle between 5 and 6 frames
a_2 = 340  # Link length a_2
a_3 = 25  # Link length a_3
d_3 = 90  # Displacement length d_3
d_4 = 260  # displacement length d_4

# Combine numerical parameters
num_params = (theta_1, theta_2, theta_3, theta_4, theta_5, theta_6, a_2, a_3, d_3, d_4)

# Get numerical D-H table
DH_numerical = dh_sym2num(*num_params)
print('Denavit-Hartenberg numerical table:')
print(DH_numerical.round(2))

Denavit-Hartenberg numerical table:
[[  0.     0.     0.     0.79]
 [ -1.57   0.     0.     1.05]
 [  0.   340.    90.     1.31]
 [ -1.57  25.   260.     0.  ]
 [  1.57   0.     0.     0.  ]
 [ -1.57   0.     0.     0.  ]]


In [13]:
# Get desired transformation matrix
i = 6  # Set desired frame index
tf = get_tf_matrix(DH_table, i)
f_tf = sp.lambdify(params, tf)
print(f'Numerical tranformation matrix between {i} and {i-1} frames:')
print(f_tf(*num_params))

Numerical tranformation matrix between 6 and 5 frames:
[[ 1. -0.  0.  0.]
 [ 0.  0.  1.  0.]
 [-0. -1.  0.  0.]
 [ 0.  0.  0.  1.]]


In [6]:
# Get resulted transformation matrix for end-effector:
f_res = sp.lambdify(params, res_T)
print(f'Transformation matrix for end-effector:')
print(f_res(*num_params))

Transformation matrix for end-effector:
[[  -0.5           0.70710678   -0.5         -85.93145751]
 [  -0.5          -0.70710678   -0.5          41.34776311]
 [  -0.70710678    0.            0.70710678 -128.27854371]
 [   0.            0.            0.            1.        ]]


In [7]:
# Get position for end-effector:
f_pos = sp.lambdify(params, pos)
print(f'End-effector position:')
print(f_pos(*num_params))

End-effector position:
[[ -85.93145751]
 [  41.34776311]
 [-128.27854371]]


**Conclusion: Denavit-Hartenberg is a powerful tool to solve forward kinematics for robotic arms. This notebook was prepared in a way to let it use in future work. Simply fill the table with parameters and set numerical parameters in the second part of the notebook.**